In [12]:
# -*- coding: utf-8 -*-
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import numpy as np
import pandas as pd

# ======================
# 0) 讀取資料
# ======================
train = pd.read_excel("train.xlsx")
test  = pd.read_excel("test.xlsx")

# 標籤欄
TARGET_COL = "崩塌"
if TARGET_COL not in train.columns:
    raise ValueError(f"找不到標籤欄：{TARGET_COL}")

# 特徵 / 標籤（先用全部數值特徵，若含非數值請先轉型或剔除）
X_full_all = train.drop(columns=[TARGET_COL])
y          = train[TARGET_COL]

# ======================
# 1) 切分與補值（以訓練集統計量 fit，再套到 val/test）
# ======================
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

X_train, X_val, y_train, y_val = train_test_split(
    X_full_all, y, stratify=y, test_size=0.2, random_state=42
)

imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_val_imp   = pd.DataFrame(imputer.transform(X_val),     columns=X_val.columns,   index=X_val.index)
X_test_imp  = pd.DataFrame(imputer.transform(test[X_full_all.columns]), columns=X_full_all.columns, index=test.index)


In [13]:
# 2) 補值後再訓練隨機森林 → 找前 30 名重要特徵
# ======================
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=50, n_estimators=200, n_jobs=-1)
rf.fit(X_train_imp, y_train)

importances   = rf.feature_importances_
feature_names = X_train_imp.columns

feat_imp = (
    pd.DataFrame({'feature': feature_names, 'importance': importances})
      .sort_values(by='importance', ascending=False)
      .reset_index(drop=True)
)

print("\n=== Top 30 Feature Importances (RandomForest on imputed data) ===")
print(feat_imp.head(30))

# 3) 取前 10 名覆寫 selected_features，並把 train/val/test 都切到這 10 欄
# ======================
top_n = 10
selected_features = feat_imp.head(top_n)['feature'].tolist()
print(f"\n>>> 使用前 {top_n} 個特徵：{selected_features}")

# 以「前十特徵」重建資料矩陣（已補值版本）
X_train_sel = X_train_imp[selected_features].copy()
X_val_sel   = X_val_imp[selected_features].copy()

# test 也以相同特徵切出；若 test 少欄位就報錯提示
missing_in_test = [c for c in selected_features if c not in X_test_imp.columns]
if missing_in_test:
    raise ValueError(f"test 缺少以下前十特徵：{missing_in_test}，請確認 test 欄位或特徵工程流程一致。")
X_test_sel  = X_test_imp[selected_features].copy()


=== Top 30 Feature Importances (RandomForest on imputed data) ===
   feature  importance
0     植生指標    0.064404
1      斷層距    0.040189
2     f_85    0.039264
3     最小曲率    0.017262
4     f_32    0.013955
5     f_48    0.013595
6     平均坡向    0.012269
7    f_199    0.011646
8    f_141    0.006675
9    順向坡指標    0.006155
10   f_108    0.005829
11   f_175    0.005557
12     f_5    0.005438
13    f_70    0.005412
14   f_164    0.005336
15   f_119    0.005301
16    f_64    0.005217
17   f_185    0.005188
18   f_126    0.005155
19   f_186    0.005120
20   f_142    0.005114
21    f_27    0.005086
22   f_151    0.005054
23   f_128    0.005047
24   f_153    0.005040
25   f_173    0.005026
26    f_80    0.005010
27     f_7    0.005009
28   f_109    0.004994
29   f_200    0.004980

>>> 使用前 10 個特徵：['植生指標', '斷層距', 'f_85', '最小曲率', 'f_32', 'f_48', '平均坡向', 'f_199', 'f_141', '順向坡指標']


In [14]:
# 2) XGBoost：單模訓練（CPU-only，使用「前十特徵 + 已補值」的 X_train_sel / X_val_sel）
# ======================
import xgboost as xgb
from sklearn.metrics import (
    roc_auc_score, average_precision_score, accuracy_score,
    classification_report, confusion_matrix, f1_score, roc_curve
)
from sklearn.model_selection import StratifiedKFold

# 類別不平衡權重（neg/pos）— 以目前訓練子集 y_train 為準
pos_cnt = (y_train == 1).sum()
neg_cnt = (y_train == 0).sum()
scale_pos_weight = (neg_cnt / max(pos_cnt, 1))

def make_xgb_params(spw=None):
    return dict(
        n_estimators=10000,
        max_depth=8,
        learning_rate=0.05,
        eval_metric="logloss",
        random_state=42,
        subsample=0.9,
        colsample_bytree=0.9,
        min_child_weight=1,
        reg_lambda=1.0,
        reg_alpha=0.0,
        scale_pos_weight=scale_pos_weight if spw is None else spw,
        tree_method="hist",   # CPU
        n_jobs=-1
    )

# 單模訓練用「已補值 + 前十特徵」
xgb_clf = xgb.XGBClassifier(**make_xgb_params())
xgb_clf.fit(X_train_sel, y_train)

# 驗證集評估（同樣用「已補值 + 前十特徵」）
y_val_proba = xgb_clf.predict_proba(X_val_sel)[:, 1]
roc = roc_auc_score(y_val, y_val_proba)
pra = average_precision_score(y_val, y_val_proba)
print(f"\nValidation ROC-AUC = {roc:.4f} | PR-AUC = {pra:.4f}")

# 找最佳 F1 門檻
best_thr, best_f1 = 0.5, 0.0
for t in np.linspace(0.0, 1.0, 201):
    y_pred = (y_val_proba >= t).astype(int)
    f1 = f1_score(y_val, y_pred, zero_division=0)
    if f1 > best_f1:
        best_thr, best_f1 = t, f1
print(f"最佳 F1={best_f1:.4f} 對應 threshold={best_thr:.4f}")

best_thr = 0.005
y_val_pred = (y_val_proba >= best_thr).astype(int)
print("\n=== Validation Report @ Best-F1 Threshold (CPU) ===")
print(f"Accuracy : {accuracy_score(y_val, y_val_pred):.4f}")
print(classification_report(y_val, y_val_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_val, y_val_pred))




Validation ROC-AUC = 0.8566 | PR-AUC = 0.5585
最佳 F1=0.5591 對應 threshold=0.3700

=== Validation Report @ Best-F1 Threshold (CPU) ===
Accuracy : 0.8030
              precision    recall  f1-score   support

           0     0.9533    0.8150    0.8787      1276
           1     0.3552    0.7182    0.4753       181

    accuracy                         0.8030      1457
   macro avg     0.6542    0.7666    0.6770      1457
weighted avg     0.8790    0.8030    0.8286      1457

Confusion matrix:
 [[1040  236]
 [  51  130]]


In [15]:
# 3) 5-fold 集成（每折各自補值＋前十特徵，CPU-only，避免資料洩漏）
import numpy as np
from sklearn.impute import SimpleImputer

# 提醒：X_full_sel 是你在「RF 篩十特徵」後，尚未補值的完整特徵表（train 的那份）
X_full_sel = train[selected_features].copy()

test_proba_blend = np.zeros(len(X_test_sel))  # 已補值後之 test（前十特徵）的長度
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_id = 1
for tr_idx, va_idx in skf.split(X_full_sel, y):
    # 分割當折資料（未補值、但只含前十特徵）
    X_tr_raw, X_va_raw = X_full_sel.iloc[tr_idx], X_full_sel.iloc[va_idx]
    y_tr,     y_va     = y.iloc[tr_idx],        y.iloc[va_idx]

    # 每折以「當折訓練子集」的統計量做補值（median）
    imp_k = SimpleImputer(strategy='median')
    X_tr  = pd.DataFrame(imp_k.fit_transform(X_tr_raw), columns=X_tr_raw.columns, index=X_tr_raw.index)
    X_va  = pd.DataFrame(imp_k.transform(X_va_raw),     columns=X_va_raw.columns, index=X_va_raw.index)
    X_te  = pd.DataFrame(imp_k.transform(X_test_sel),   columns=X_test_sel.columns, index=X_test_sel.index)

    # 每折調整類別權重（neg/pos）
    spw = (y_tr == 0).sum() / max((y_tr == 1).sum(), 1)

    # 純 CPU 訓練
    mdl = xgb.XGBClassifier(**make_xgb_params(spw))
    mdl.fit(X_tr, y_tr)

    # 折內驗證觀察（可略）
    oof = mdl.predict_proba(X_va)[:, 1]
    fold_roc = roc_auc_score(y_va, oof)
    fold_pr  = average_precision_score(y_va, oof)
    print(f"[Fold {fold_id}] ROC-AUC={fold_roc:.4f} | PR-AUC={fold_pr:.4f}")
    fold_id += 1

    # 對 test 累加平均機率（以每折的 imputer 轉換過的 X_te）
    test_proba_blend += mdl.predict_proba(X_te)[:, 1] / skf.n_splits
#     # --- 初始化 ---
# test_proba_blend = np.zeros(len(X_te))
# fold_pr_scores = []  # 儲存每折的 PR-AUC，稍後做加權平均

# for fold_id, (tr_idx, va_idx) in enumerate(skf.split(X_tr, y_tr), 1):
#     X_tr_f, X_va_f = X_tr.iloc[tr_idx], X_tr.iloc[va_idx]
#     y_tr_f, y_va_f = y_tr.iloc[tr_idx], y_tr.iloc[va_idx]

#     mdl = xgb.XGBClassifier(**make_xgb_params(spw))
#     mdl.fit(X_tr_f, y_tr_f)

#     oof = mdl.predict_proba(X_va_f)[:, 1]
#     fold_roc = roc_auc_score(y_va_f, oof)
#     fold_pr  = average_precision_score(y_va_f, oof)
#     fold_pr_scores.append(fold_pr)
#     print(f"[Fold {fold_id}] ROC-AUC={fold_roc:.4f} | PR-AUC={fold_pr:.4f}")

#     # --- 暫存每折 test 預測 ---
#     fold_pred = mdl.predict_proba(X_te)[:, 1]
#     if fold_id == 1:
#         all_fold_preds = [fold_pred]
#     else:
#         all_fold_preds.append(fold_pred)

# # --- 將各折的 test 預測加權平均 ---
# fold_pr_scores = np.array(fold_pr_scores)
# weights = fold_pr_scores / fold_pr_scores.sum()  # 正規化成權重
# test_proba_blend = np.average(np.vstack(all_fold_preds), axis=0, weights=weights)

# print("\n🎯 各折 PR-AUC 權重：", np.round(weights, 3))
# print("✅ 已以 PR-AUC 最高折數給予更高權重完成加權平均。")


[Fold 1] ROC-AUC=0.8904 | PR-AUC=0.6586
[Fold 2] ROC-AUC=0.8436 | PR-AUC=0.5927
[Fold 3] ROC-AUC=0.8703 | PR-AUC=0.6141
[Fold 4] ROC-AUC=0.8946 | PR-AUC=0.6335
[Fold 5] ROC-AUC=0.8635 | PR-AUC=0.5622


In [25]:
# ---- 參數區 ----
THRESHOLD = 0.006          # 固定門檻（沿用你的原始值）
MATCH_TARGET_COUNT = False    # 想匹配陽性數量時改 True
TARGET_POS = 500              # 預期 test 中的 1 的數量
# ----------------

y_test_proba = test_proba_blend

y_test_pred = (y_test_proba >= THRESHOLD).astype(int)
print(f"[Info] 使用固定門檻 threshold={THRESHOLD}")

# 統計
from collections import Counter
cnt = Counter(y_test_pred.tolist())
print("Prediction label counts:", cnt)

# 若 test 有 ID 欄位就沿用，否則 0..n-1
if "ID" in test.columns:
    id_series = test["ID"].reset_index(drop=True)
else:
    id_series = pd.Series(np.arange(len(y_test_pred)), name="ID")

submission = pd.DataFrame({"ID": id_series, "Label": y_test_pred.astype(int)})
submission.to_csv("submission_first.csv", index=False, encoding="utf-8-sig")
print("Saved: submission_first.csv ✅")


[Info] 使用固定門檻 threshold=0.006
Prediction label counts: Counter({1: 546, 0: 454})
Saved: submission_first.csv ✅
